SQL fun - IMDB querying and visualisation

My notebook queries the IMDb database to explore and visualise the IMDb dataset using pandas and matplotlib. The database itself is large,and through exploration of the IMDb dataset, I will practice querying the database using python, then to process and visualise the retrieved data with the pandas package. I am a movie buff, so I intend on exploring these questions:

What are the average ratings for the TV show 'The X-files'?
What movies have the highest soundtrack rating?
What genres are there?
How many movies are there in each genre?
How many movies are made in each genre each year?
How do the average ages of leading actors and actresses compare in each genre?
What is a typical runtime for movies in each genre?

Imports

In [18]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt # general plotting
import matplotlib as mpl # for figure quality (dpi)
import os

My Formatting

In [19]:
# Fontsize
fs = 12

# Figure quality
mpl.rcParams['figure.dpi']= 100 # in notebook resolution
mpl.rc("savefig", dpi=400) # higher resolution when saving

# Set font
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica'

# Set the style of the axes and the text color
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']= 1
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'
plt.rcParams['text.color']='#333F4B'

Image Folder for Plots

In [20]:
import os

# Create directory if it doesn't already exist
img_directory = './images/plots'
if not os.path.exists(img_directory):
    os.makedirs(img_directory)  # <-- This will create parent dirs if needed
    print("Directory:", img_directory, "created.")


Connect to my DB file

In [21]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
ALTER USER 'your_username'@'localhost' IDENTIFIED WITH mysql_native_password BY 'your_password';
FLUSH PRIVILEGES;

SyntaxError: invalid syntax (287973027.py, line 1)

In [ ]:
import pandas as pd

imdb = pd.read_csv("Data\imdb_top_1000.csv")

<>:3: SyntaxWarning: invalid escape sequence '\i'
<>:3: SyntaxWarning: invalid escape sequence '\i'
C:\Users\profe\AppData\Local\Temp\ipykernel_43184\78103782.py:3: SyntaxWarning: invalid escape sequence '\i'
  imdb = pd.read_csv("Data\imdb_top_1000.csv")


In [ ]:
imdb

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


In [35]:
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
import sqlalchemy as sqla

def get_uri_from_env():
    user = os.getenv("MYSQL_USER")  
    password = os.getenv("MYSQL_PASSWORD")
    host = os.getenv("MYSQL_HOST")
    port = os.getenv("MYSQL_PORT")
    db = os.getenv("MYSQL_DB")
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'

uri = get_uri_from_env()

engine = sqla.create_engine(uri)

In [24]:
imdb.to_sql("imdbdb", if_exists="replace",con=engine)

1000

In [45]:
insights = pd.read_csv("Data\\title.principals.tsv\\title.principals.tsv", sep="\t")

In [48]:
insights.shape

(93759105, 6)

In [49]:
insights.iloc[:50000000,:].to_sql("title_principals", if_exists="replace", con=engine)




StatementError: (builtins.MemoryError) 
[SQL: INSERT INTO title_principals (`index`, tconst, ordering, nconst, category, job, characters) VALUES (%(index)s, %(tconst)s, %(ordering)s, %(nconst)s, %(category)s, %(job)s, %(characters)s)]
[parameters: [{'index': 0, 'tconst': 'tt0000001', 'ordering': 1, 'nconst': 'nm1588970', 'category': 'self', 'job': '\\N', 'characters': '["Self"]'}, {'index': 1, 'tconst': 'tt0000001', 'ordering': 2, 'nconst': 'nm0005690', 'category': 'director', 'job': '\\N', 'characters': '\\N'}, {'index': 2, 'tconst': 'tt0000001', 'ordering': 3, 'nconst': 'nm0005690', 'category': 'producer', 'job': 'producer', 'characters': '\\N'}, {'index': 3, 'tconst': 'tt0000001', 'ordering': 4, 'nconst': 'nm0374658', 'category': 'cinematographer', 'job': 'director of photography', 'characters': '\\N'}, {'index': 4, 'tconst': 'tt0000002', 'ordering': 1, 'nconst': 'nm0721526', 'category': 'director', 'job': '\\N', 'characters': '\\N'}, {'index': 5, 'tconst': 'tt0000002', 'ordering': 

In [ ]:
insights.iloc[50000000,:]to_sql("title_principals", if_exists="append", con=engine)

In [46]:
insights

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N
...,...,...,...,...,...,...
93759100,tt9916880,17,nm0996406,director,principal director,\N
93759101,tt9916880,18,nm1482639,writer,\N,\N
93759102,tt9916880,19,nm2586970,writer,books,\N
93759103,tt9916880,20,nm1594058,producer,producer,\N


NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported

In [ ]:
# import mysql.connector

# # Establish the connection
# conn = mysql.connector.connect(
#     host="localhost",          # Change if using remote server
#     user="your_username",      # Your MySQL username
#     password="GeoRunz",  # Your MySQL password
#     database="imdb_insights"   # The database name
# )

# # Create a cursor object to execute queries
# cursor = conn.cursor()

# # Test query to ensure connection works
# cursor.execute("SHOW TABLES;")

# # Fetch and print table names
# for table in cursor.fetchall():
#     print(table)

# # Always close connection when done
# cursor.close()
# conn.close()

NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported

In [ ]:
tables = cursor.fetchall()
print('IMDb Database contains the following tables:')
print('--------------------------------------------')
for table in tables:
    print(table[0])

NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported

Q1 - Visualizing the Brilliant "The X-Files"

What is the average rating of each episode of The X-Files?


In [ ]:
Query1 = """SELECT E.season_number, E.episode_number, T2.primary_title, R.average_rating
FROM Titles AS T1, Titles AS T2, Episode_belongs_to AS E, Title_ratings AS R
WHERE T1.primary_title = 'The X-Files'
AND T1.title_type = 'tvSeries'
AND T1.title_id = E.parent_tv_show_title_id
AND T2.title_type = 'tvEpisode'
AND T2.title_id = E.episode_title_id
AND T2.title_id = R.title_id
ORDER BY E.season_number, E.episode_number;"""
cursor.execute(Query1)
data = cursor.fetchall()
df = pd.DataFrame(data,columns=['season_no','episode_no','ep_title','avg_rating'])
df.head()

How many episodes were there in The X-Files per season? 
And what was the average of the average episode ratings for each season?

In [ ]:
Query2 = """SELECT Q22.season_number, COUNT(*) AS Number_of_episodes, AVG(Q22.average_rating) AS Average_of_ep_average_ratings
FROM Q22
GROUP BY Q22.season_number
ORDER BY Q22.season_number;"""
cursor.execute(Query2)
data = cursor.fetchall()
df2 = pd.DataFrame(data,columns=['season_no','no_of_eps','avg_of_avg_ratings'])
df2.head()

In [ ]:
# Imports